reference https://www.kaggle.com/gaoyuan19930220/xgboost

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import os
import sklearn
import xgboost as xgb
import operator
import optuna

from functools import partial
from matplotlib import pylab as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold


import warnings
warnings.filterwarnings('ignore')
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [3]:
count_itr = 0

def objective_optuna(X, y, trial):
    global count_itr
    print(count_itr, end=' ')
    count_itr += 1
    #最適化するパラメータを指定
    params = {
        'n_estimators': trial.suggest_int('n_estimators',1,1000),
        'gamma': trial.suggest_uniform('gamma',0,1),
        'reg_lambda': trial.suggest_uniform('lambda',0,2),
        'learning_rate': trial.suggest_uniform('learning_rate',0,0.3),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree',0,1),
        'min_child_weight': trial.suggest_uniform('min_child_weight',0,10),
#         'device':'cpu',
#         'gpu_id':0,
        'tree_method':'gpu_hist'
    }
    
    #モデルを定義
    model = xgb.XGBClassifier(random_state=2,**params)

#     (train_X, test_X ,train_y, test_y) = train_test_split(X, y, test_size = 0.3, random_state = 2)
    
#     #3-foldクロスバリデーション
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=2)
    scores = cross_validate(model, X=X, y=y, cv=kf, scoring='roc_auc')
    
    return 1.0 - scores['test_score'].mean()

In [4]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [5]:
#csvデータの呼び出し場所
loadpath = "../input/"
#csvデータの保存場所
savepath = "../output/"

In [6]:
train = pd.read_csv(loadpath + "train.csv")
test = pd.read_csv(loadpath + "test.csv")

train.set_index('ID_code',inplace=True)
test.set_index('ID_code',inplace=True)

train_index = train.index
test_index = test.index

In [7]:
labels = train.target
train.drop('target', axis=1, inplace=True)
#データを分割
#(train_X, test_X ,train_y, test_y) = train_test_split(train_X, train_y, test_size = 0.3, random_state = 666)

In [8]:
# xgboost.callback.record_evaluation

optunaでハイパーパラメーター決定

In [15]:
%%time
f = partial(objective_optuna, train, labels)
study = optuna.create_study()
study.optimize(f, n_trials=10)

0 

KeyboardInterrupt: 

In [9]:
# train a xgboost classifier
# params = {"num_class": 2}
# train_xgb = xgb.DMatrix(train, labels)
# test_xgb  = xgb.DMatrix(test)
# trainRound = 100
# gbm = xgb.train(params, train_xgb, trainRound)
# pred = gbm.predict(test_xgb)
# pred = pred.astype('int')

In [10]:
study.best_params

NameError: name 'study' is not defined

In [11]:
kf = StratifiedKFold(n_splits=10, shuffle=True,)
scores = cross_validate(model, X=train, y=labels, cv=kf, scoring='accuracy')
scores['test_score'].mean()

NameError: name 'model' is not defined

In [26]:
%%time
study_params={'n_estimators': 937, 'gamma': 0.6007567248088769, 'lambda': 0.6926715962837825, 'learning_rate': 0.13391942651471928, 'max_depth': 2, 'subsample': 0.3285581618280114, 'colsample_bytree': 0.37375086250914463, 'min_child_weight': 4.9408427396915275}
    
#モデルの定義
model = xgb.XGBClassifier(random_state=2,tree_method='gpu_exact',**study_params)
model.fit(train, labels)

CPU times: user 45.1 s, sys: 18 s, total: 1min 3s
Wall time: 1min 3s


In [27]:
%%time
study_params={'n_estimators': 937, 'gamma': 0.6007567248088769, 'lambda': 0.6926715962837825, 'learning_rate': 0.13391942651471928, 'max_depth': 2, 'subsample': 0.3285581618280114, 'colsample_bytree': 0.37375086250914463, 'min_child_weight': 4.9408427396915275}
    
#モデルの定義
model = xgb.XGBClassifier(random_state=2,**study_params)
model.fit(train, labels)

CPU times: user 6min 25s, sys: 1.01 s, total: 6min 26s
Wall time: 6min 26s


In [17]:
pred = model.predict_proba(test)[:,1]

In [24]:
# create submission file
submission_result = pd.DataFrame({"target":pred},index=test.index)
submission_result.to_csv(savepath + 'submission_xgb.csv',index='False')

In [17]:
type(pred)

numpy.ndarray

# 以下適当なコード

In [13]:
submission_result.head()

,target
ID_code,
test_0,0
test_1,0
test_2,0
test_3,0
test_4,0


In [40]:
type(submission_result.target)

pandas.core.series.Series

In [47]:
a = submission_result.target

In [46]:
labels.value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [43]:
a.value_counts()

0    197961
1      2039
Name: target, dtype: int64

In [ ]:
labels.

In [37]:
submission_result.target

AttributeError: 'Series' object has no attribute 'value_count'

In [ ]:
#重要度のやつ


features = train.columns
# create a feature map
outfile = open('xgb.fmap', 'w')
i = 0
for feat in features:
    outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    i = i + 1
outfile.close()

# plot feature importance
importance = gbm.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))
df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()
plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 20))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')



In [31]:

#使うデータの前処理
train_xgb = xgb.DMatrix(df.drop('Survived', axis=1), df.Survived)
test_xgb  = xgb.DMatrix(df_test)
    
#最適化するパラメータを設定
params = {
     'n_estimators': 100,
     'gamma': 0,
     'reg_lambda': 1,
      #'max_depth': trial.suggest_int('max_depth', 2, 32)
}
    
#モデルの定義
model = xgb.XGBClassifier(random_state=2,**params)
model.fit(df.drop('Survived', axis=1), df.Survived)
    
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
# scores = cross_validate(model, X=X, y=y, cv=kf, scoring='accuracy')
    
# return 1.0 - scores['test_score'].mean()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=2,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)